# Берлин Влада 202
# Домашнее задание 2

In [1]:
from natasha import (
    Segmenter,
    MorphVocab,

    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,

    PER,
    NamesExtractor,

    Doc
)

import spacy
from pymorphy2 import MorphAnalyzer
from sklearn.metrics import accuracy_score

In [2]:
with open('corpus.txt', encoding='utf8') as f:
    text = f.read()

In [3]:
nlp = spacy.load('ru_core_news_sm')

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)
doc = Doc(text)

morph = MorphAnalyzer()

слова, которые, на мой взгляд, могли вызвать сложности: мгу, прям, уггга, завкафедрой, юрфака, побежу, победю, ран, лучче, трепков, житьё-бытьё, повтикали, спбгу, спбпу, ниу вшэ, итмо, скроллинг, наркодиспансер, телеграм-канал, рунета

1. такие штуки как мгу, ран - это аббревиатуры, которые могут быть распознаны как глагол (мгу - нач. форма мгить)
2. прям - разговорное слово, которое может получить неверный тег
3. лучче - честно говоря, не знаю, опечатка это или нет, но это слово явно должно вызвать затруднения у программы
4. побежу, победю - разговорные фактически аграмматические формы глагола
5. повтикали - глагол, который может вызвать затруднение, тк это явно какое-то диалектное слово (ну или слово другого родственного языка вроде украинского)
6. скроллинг, телеграм-канал, рунет - неологизмы, которые не факт, что программа правильно разспознает

ТЭГИ с https://universaldependencies.org/u/pos/
но с некоторыми отличиями (например, я не различала CCONJ и SCONJ, у меня просто CONJ)

я взяла именно этот набор тегов, так как он наиболее универсальный и удобный (к тому же этот набор используют два из трех выбранных мной теггеров)

работаем с natasha, pymorphy, spacy

проверка на то, одинаковая ли длина у автоматической и ручной разметок, если нет, то делаем так, чтобы она была одинаковой

In [4]:
def checking(man, auto):
    m = man.copy()
    a = auto.copy()
    for i in range(len(m)):
        if m[i][0] != a[i][0]:
            if len(m[i][0]) > len(a[i][0]):
                m.insert(i+1, m[i][0])
                i += 1
            else:
                a.insert(i+1, a[i][0])
                i += 1
    return m, a

оставляем только части речи, чтобы сделать проверку на accuracy

In [19]:
def only_pos(man, auto, dct):
    m_pos = []
    a_pos = []
    for i in range(len(man)):
        if auto[i][1] in dct.keys():
            m_pos.append(man[i][1])
            a_pos.append(dct[auto[i][1]])
        else:
            m_pos.append(man[i][1])
            a_pos.append(auto[i][1])
    return m_pos, a_pos

In [6]:
man_ann = []
with open('corpus_MAN.txt', encoding='utf8') as f:
    t = f.read()

toks = t.split(' ')
for tok in toks:
    man_ann.append([tok.split('_')[0], tok.split('_')[1]])

## natasha

словарь перевода теггов

In [7]:
nat_to_my = {'AUX': 'VERB', 'CCONJ': 'CONJ', 'SCONJ': 'CONJ'}

обработка

In [8]:
nat_ann = []

doc.segment(segmenter)
doc.tag_morph(morph_tagger)

for token in doc.tokens:
  token.lemmatize(morph_vocab)
    
for el in doc.tokens:
    nat_ann.append([el.text, el.pos])

сравнение ручной и автоматической разметок

In [20]:
man_nat, nat = checking(man_ann, nat_ann)

m_n_pos, nat_pos = only_pos(man_nat, nat, nat_to_my)

print("Accuracy: %.4f" % accuracy_score(m_n_pos, nat_pos))

Accuracy: 0.9173


## pymorphy

словарь перевода теггов

In [10]:
pymorphy_to_my = {'ADJF': 'ADJ', 'ADJS': 'ADJ', 'COMP': 'ADJ', 'INFN': 'VERB', 'PRTF': 'VERB', 'PRTS': 'VERB',
'GRND': 'VERB', 'NUMR': 'NUM', 'ADVB': 'ADV', 'NPRO': 'PRON', 'PRED': 'ADV', 'PREP': 'ADP', 'PRCL': 'PART'}

обработка

In [11]:
pm_ann = []

for token in doc.tokens:
    if morph.parse(token.text)[0].tag.POS:
        pm_ann.append([token.text, morph.parse(token.text)[0].tag.POS])
    else:
        pm_ann.append([token.text, 'PUNCT'])

сравнение ручной и автоматической разметок

In [21]:
man_pm, pm = checking(man_ann, pm_ann)

m_pm_pos, pm_pos = only_pos(man_pm, pm, pymorphy_to_my)

print("Accuracy: %.4f" % accuracy_score(m_pm_pos, pm_pos))

Accuracy: 0.8710


## spacy

Так как natasha использует те же теги, что и spacy, список для перевода тегов остаётся тем же

обработка

In [23]:
sp_ann = []

for token in doc.tokens:
    sp_ann.append([token.text, token.pos])

сравнение ручной и автоматической разметок

In [24]:
man_sp, sp = checking(man_ann, sp_ann)

m_sp_pos, sp_pos = only_pos(man_sp, sp, nat_to_my)

print("Accuracy: %.4f" % accuracy_score(m_sp_pos, sp_pos))

Accuracy: 0.9173


Получается, что natasha и spacy справились одинаково хорошо, pymorphy немного похуже.

In [25]:
print("Accuracy: %.4f" % accuracy_score(nat_pos, sp_pos))

Accuracy: 1.0000


Как можно заметить, natasha и spacy разметили корпус абсолютно одинако. Я буду использовать natasha по личным предпочтениям.

1. очень + ADJ
2. не + VERB
3. ADJ + NOUN

я выбрала эти ngramm'ы, так как они, на мой взгляд, должны неплохо помогать указывать на полярность отзыва, например, очень + прилагательное будет, скорее всего, может неплохо помочь, так как такие пары должны хорошо указывать на тональность

In [56]:
def chunker(text):
  ngramms = []
  dct = {'ADJF': 'ADJ', 'ADJS': 'ADJ', 'COMP': 'ADJ', 'INFN': 'VERB',
  'PRTF': 'VERB', 'PRTS': 'VERB', 'GRND': 'VERB'}

  doc = Doc(text)
  doc.segment(segmenter)
  doc.tag_morph(morph_tagger)

  for token in doc.tokens:
    token.lemmatize(morph_vocab)

  i = 0
  l = -2  ## На случай, если какой-то из интересующих нас частеречных тегов, являющийся вторым элементом ngramm'ы, появится раньше, чем первый элемент такой ngramm'ы
  k = -2
  m = -2

  for t in doc.tokens:

    if t.pos == 'VERB':
      if i == m + 1:              ## Если предыдущее слово является элементом ngramm'ы
        mor = morph.parse(t.text)
        for el in mor:
          if el.tag.POS in dct:
            tg = dct[el.tag.POS]
          else:
            tg = el.tag.POS
          if tg == 'VERB':
            ngramms[-1].append(el.normal_form)  ## В наших интересах записывать начальные формы, так как в таком случае от них будет больше пользы (если мы говорим о целях, которые мы преследовали в первой домашке)
            break
          if el == mor[-1]:             ## Если pymorphy не смог разобрать слово как нужную нам часть речи, то приходится добавлять форму, которая встретилась в тексте
            ngramms[-1].append(t.text)

    elif t.pos == 'NOUN':
      if i == k + 1:              ## Если предыдущее слово является элементом ngramm'ы
        mor = morph.parse(t.text)
        for el in mor:
          if el.tag.POS == 'NOUN':
            ngramms[-1].append(el.normal_form)
            break
          if el == mor[-1]:
            ngramms[-1].append(t.text)

    elif t.pos == 'ADJ':
      if i == l + 1:              ## Если предыдущее слово является элементом ngramm'ы
        mor = morph.parse(t.text)
        for el in mor:
          tg = ''
          if el.tag.POS in dct:   ## Так как все теги, обозначающие прилагательные, записаны в словаре перевода тегов, можно проверять подходит ли нам этот разбор только по тем тегам, которые есть в словаре
            tg = dct[el.tag.POS]
          if tg == 'ADJ':
            ngramms[-1].append(el.normal_form)
            break
          if el == mor[-1]:
            ngramms[-1].append(t.text)
            
    elif k == i - 1 or l == i - 1 or m == i - 1:
      ngramms = ngramms[:-1]

    if t.pos == 'ADJ':
      k = i
      ngramms.append([t.text])

    elif t.text == 'очень':
      l = i
      ngramms.append([t.text])

    elif t.text == 'не':
      m = i
      ngramms.append([t.text])

    i += 1
  return ngramms